In [1]:
import torch
import os
from torchvision.transforms import ToTensor,Resize, Normalize,Compose, ToPILImage
from torch.utils.data import Dataset, DataLoader
from tqdm import *
import matplotlib.pyplot as plt

c:\Users\Wen2Tee5\Desktop\machine learning\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCH = 20
BATCH_SIZE = 32

In [3]:
from PIL import Image
class FlowerDataset(Dataset):

    def __init__(self, root_path, split = 'train', transform = None, device = 'cpu') -> None:
        super(FlowerDataset, self).__init__()
        self.transform = transform
        self.root_path = os.path.join(root_path, split)
        self.class_list = os.listdir(self.root_path)
        self.datas = []
        self.device = device
        for label, class_name in enumerate(self.class_list):
            classPath = os.path.join(self.root_path, class_name)
            img_list = os.listdir(classPath)
            for imgName in img_list:
                imgPath = os.path.join(classPath, imgName)
                self.datas.append((label, imgPath))
    
    def __len__(self):
        return len(self.datas)
    
    def __getitem__(self, index):
        label, imgPath = self.datas[index]
        label = torch.tensor(data=label)
        img = Image.open(imgPath)
        if self.transform is not None:
            img = self.transform(img)
        return label.to(self.device), img.to(self.device)


In [4]:
root_path = '.\\dataset\\flowers_\\'

In [5]:
show_dataset = FlowerDataset(
        root_path=root_path, 
    split='train', 
    transform=Compose(
        [
        Resize(size=(150, 150)),
        ToTensor(),
        ]
        ),
        device=DEVICE
    )

In [12]:
ToImg = ToPILImage()

In [5]:
y,x = show_dataset[1599]
print(f'y = {y}')
print(f'Class is {show_dataset.class_list[y.item()]}')
print(f'x shape = {x.shape}')
img = ToImg(x)
plt.imshow(img)
plt.show()

NameError: name 'show_dataset' is not defined

In [6]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [7]:

train_dataset = FlowerDataset(
    root_path=root_path, 
    split='train', 
    transform=Compose([
        Resize(size=(150, 150)),
        ToTensor(),
        Normalize(mean, std)
        ]),
        device=DEVICE
    )
test_dataset = FlowerDataset(
        root_path=root_path, 
        split='test',
        transform=Compose([
        Resize(size=(150, 150)),
        ToTensor(),
        Normalize(mean, std)
        ]),
        device=DEVICE)


In [8]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size = BATCH_SIZE, shuffle=True)

In [9]:
class Network(torch.nn.Module):

    def __init__(self) -> None:
        super(Network, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_1 = torch.nn.Linear(3 * 150 * 150, 256)
        self.linear_2 = torch.nn.Linear(256, 128)
        self.linear_3 = torch.nn.Linear(128, 64)
        self.linear_4 = torch.nn.Linear(64, 5)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        x = self.linear_3(x)
        x = self.linear_4(x)      
        return x

In [10]:
class CNNBased(torch.nn.Module):

    def __init__(self) -> None:
        super(CNNBased, self).__init__()
        self.Conv1 = torch.nn.Conv2d(3, 96, kernel_size=3)
        self.Conv2 = torch.nn.Conv2d(96, 768, kernel_size=3)
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        self.linear1 = torch.nn.Linear(768 * 36 * 36, 64)
        self.linear2 = torch.nn.Linear(64, 5)
        self.flatten = torch.nn.Flatten()
    
    def forward(self, x):
        x = self.Conv1(x)
        x = self.maxPool(x)
        x = self.Conv2(x)
        x = self.maxPool(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [11]:
class CNN1(torch.nn.Module):

    def __init__(self) -> None:
        super(CNN1, self).__init__()
        self.Conv1 = torch.nn.Conv2d(3, 96, kernel_size=3)
        self.Conv2 = torch.nn.Conv2d(96, 768, kernel_size=3)
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        self.linear1 = torch.nn.Linear(768 * 36 * 36, 64)
        self.linear2 = torch.nn.Linear(64, 5)
        self.flatten = torch.nn.Flatten()
        self.dropout = torch.nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.Conv1(x)
        x = self.dropout(x)
        x = self.maxPool(x)
        x = self.Conv2(x)
        x = self.dropout(x)
        x = self.maxPool(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [12]:
def train_step(model, loss_fn, optimizer, dataloader):
    model.train()
    for epoch in range(EPOCH):
        print(f'Current Epoch is {epoch + 1}')
        for y, x in tqdm(dataloader):
            optimizer.zero_grad()
            pred = model(x)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()
        print(f'Current loss = {loss.item()}')



def test_step(model, dataloader):
    correct = 0.
    model.eval()
    for y, x in dataloader:
        pred = model(x)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    print(f'Total data {len(dataloader.dataset)}')
    print(f'Correct = {correct}')
    acc = correct/len(dataloader.dataset)
    print(f'The acc is {acc*100.}%')

In [13]:
cnn = CNNBased().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=1e-3)

In [14]:
train_step(cnn, loss_fn, optimizer, train_dataloader)
test_step(cnn, train_dataloader)
test_step(cnn, test_dataloader)

Current Epoch is 1


100%|██████████| 95/95 [00:11<00:00,  8.02it/s]


Current loss = 8.923250198364258
Current Epoch is 2


100%|██████████| 95/95 [00:09<00:00, 10.14it/s]


Current loss = 0.7392995953559875
Current Epoch is 3


...

Current loss = 0.0003483859181869775
Current Epoch is 19


100%|██████████| 95/95 [00:09<00:00, 10.04it/s]


Current loss = 0.0003842453588731587
Current Epoch is 20


100%|██████████| 95/95 [00:09<00:00, 10.08it/s]


Current loss = 0.0005678992602042854
Total data 3027
Correct = 3027.0
The acc is 100.0%
Total data 1296
Correct = 661.0
The acc is 51.00308641975309%


In [15]:
fcnet = Network().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcnet.parameters(), lr=1e-3)

In [16]:
train_step(fcnet, loss_fn, optimizer, train_dataloader)
test_step(fcnet, train_dataloader)
test_step(fcnet, test_dataloader)

Current Epoch is 1


100%|██████████| 95/95 [00:05<00:00, 18.91it/s]


Current loss = 1.6060676574707031
Current Epoch is 2


100%|██████████| 95/95 [00:05<00:00, 18.73it/s]


Current loss = 1.1657012701034546
Current Epoch is 3


...

Current loss = 0.5675757527351379
Current Epoch is 19


100%|██████████| 95/95 [00:04<00:00, 19.01it/s]


Current loss = 0.5793566703796387
Current Epoch is 20


100%|██████████| 95/95 [00:05<00:00, 18.67it/s]


Current loss = 0.22797584533691406
Total data 3027
Correct = 2447.0
The acc is 80.8391146349521%
Total data 1296
Correct = 461.0
The acc is 35.57098765432099%


In [17]:
cnn_wo = CNN1().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_wo.parameters(), lr=1e-3)

In [18]:
train_step(cnn_wo, loss_fn, optimizer, train_dataloader)
test_step(cnn_wo, train_dataloader)
test_step(cnn_wo, test_dataloader)

Current Epoch is 1


100%|██████████| 95/95 [00:09<00:00,  9.58it/s]


Current loss = 2.365286350250244
Current Epoch is 2


100%|██████████| 95/95 [00:09<00:00,  9.66it/s]


Current loss = 3.4932198524475098
Current Epoch is 3


...

Current loss = 4.371170143713243e-05
Current Epoch is 19


100%|██████████| 95/95 [00:10<00:00,  9.47it/s]


Current loss = 5.333029093890218e-07
Current Epoch is 20


100%|██████████| 95/95 [00:09<00:00,  9.51it/s]


Current loss = 0.11272396892309189
Total data 3027
Correct = 1994.0
The acc is 65.87380244466469%
Total data 1296
Correct = 596.0
The acc is 45.98765432098765%


In [19]:
class one_layerCNN(torch.nn.Module):

    def __init__(self) -> None:
        super(one_layerCNN, self).__init__()
        self.conv = torch.nn.Conv2d(3, 9, kernel_size=3)
        self.linear = torch.nn.Linear(74*74*9, 5)
        self.maxpool = torch.nn.MaxPool2d(2, 2)
        self.flatten = torch.nn.Flatten()
        self.dropout = torch.nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.dropout(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.linear(x)
        x = self.dropout(x)
        return x

In [20]:
temp = one_layerCNN().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(temp.parameters(), lr=1e-3)

In [21]:
train_step(temp, loss_fn, optimizer, train_dataloader)
test_step(temp, train_dataloader)
test_step(temp, test_dataloader)

Current Epoch is 1


100%|██████████| 95/95 [00:05<00:00, 18.41it/s]


Current loss = 2.775486707687378
Current Epoch is 2


100%|██████████| 95/95 [00:05<00:00, 18.31it/s]


Current loss = 1.509588599205017
Current Epoch is 3


...

Current loss = 0.729246973991394
Current Epoch is 19


100%|██████████| 95/95 [00:05<00:00, 18.48it/s]


Current loss = 1.1985636949539185
Current Epoch is 20


100%|██████████| 95/95 [00:05<00:00, 18.72it/s]


Current loss = 0.5510271787643433
Total data 3027
Correct = 1604.0
The acc is 52.98975883713247%
Total data 1296
Correct = 465.0
The acc is 35.879629629629626%
